In [28]:
import pandas as pd
import numpy as np
import IPython.display as Image

Загружаем данные из csv-файлов в DataFrame'ы сектора и категории_секторов

In [29]:
sectors = pd.read_csv('/home/python/python/DataSet/data/sectors.csv')
sector_categories = pd.read_csv('/home/python/python/DataSet/data/sector_categories.csv')

Проверяем загрузку в DataFrame сектор 

In [30]:
sectors.head(3)

,category,code,voluntary_code,name_en,description_en,name_fr,description_fr
0,111,11110,NaN,Education policy and administrative management,"Education sector policy, planning and programm...",Politique de l’éducation et gestion administra...,"Politique de l’éducation, planification et pro..."
1,111,11120,NaN,Education facilities and training,"Educational buildings, equipment, materials; s...",Equipements scolaires et formation,"Bâtiments scolaires, équipement, fournitures ;..."
2,111,11130,NaN,Teacher training,Teacher education (where the level of educatio...,Formation des enseignants,Éducation des enseignants (quand le niveau d’é...


Проверяем загрузку в DataFrame категории_секторов 

In [31]:
sector_categories.head(3)

,code,name_en,name_fr
0,110,Education,Education
1,111,"Education, Level Unspecified","Education, Niveau non spécifié"
2,112,Basic Education,Education de Base


Объединяем два DataFrame'а и проверяем результат объединенния 

In [32]:
df = sector_categories.merge(sectors, left_on='code', right_on='category', how='inner').head(100)
df.dropna() # удаляем строки с NaN
df.head(3)

,code_x,name_en_x,name_fr_x,category,code_y,voluntary_code,name_en_y,description_en,name_fr_y,description_fr
0,111,"Education, Level Unspecified","Education, Niveau non spécifié",111,11110,NaN,Education policy and administrative management,"Education sector policy, planning and programm...",Politique de l’éducation et gestion administra...,"Politique de l’éducation, planification et pro..."
1,111,"Education, Level Unspecified","Education, Niveau non spécifié",111,11120,NaN,Education facilities and training,"Educational buildings, equipment, materials; s...",Equipements scolaires et formation,"Bâtiments scolaires, équipement, fournitures ;..."
2,111,"Education, Level Unspecified","Education, Niveau non spécifié",111,11130,NaN,Teacher training,Teacher education (where the level of educatio...,Formation des enseignants,Éducation des enseignants (quand le niveau d’é...


Текущая строка разбивается на колонки. Каждая колонка обрабатывается отдельно. Для обработки колонки реализован класс Column.
    Каждая колонка содержит множество setsDict состоящее из всех слов, которые встречались в колонке.
    Производить выборку со множества setsDict невозможно. Поэтому вводим словарь mapIndexSets содержащий индексы слов включенных в множество setsDict.
    Основная обработка проводится над массивом словарей mapDictCorr. Массив содержит информацию о корреляции данного элемента колонки с элементами других колонок. Каждому элементу массива соответствует своя колонка. Элемент массива - многоуровневый словарь. Элементами словаря являются структуры описывающие корреляцию между элементом рассмативаемой колонки и элементами остальных колонк (Другая колонка: (Моя колонка: число совпадений)). Ключом внешнего словаря служат слова (элементы) других колонок. Моя колонка, которая является элементом множества колонки setsDict, является ключом к значению (value) числа совпадений (внутренний словарь). В принципе безразличен порядок ключей в элементах массива. Однако, если первым ключом будет элемент множества колонки, то все транзакции, кроме последней, будут затерты. Для ключа из элементов строки все будет ОК. Многоярусная структура объясняется отсутствием в Pyhton структур для трех измерений.

In [33]:
class Column:
    powerSets = 0
    index = 0
   
    
    def __init__(self, numberColumns):
        """Constructor"""
        # множество слов для данной колонки
        self.setsDict = set()
        #self.setsDict2 = [set() for x in range(2)]  #массив списков(list)
        # словарь последненго номера строки в df для данного слова
        self.mapColNum    = {}          
        # словарь индексов данного слова в множестве self.setsDict 
        self.mapIndexSets = {}           
        # список(массив) словарей сочетаний слова в данной колонке
        # со словами в соседних колонках
        self.mapDictCorr = [] # [dict() for x in range(20)]   #словарь (map)
        #это list из set'ов
        #self.columnCombination = [set() for i in range(numberColumns)]
        self.columnCombination = []  #список  (list)
        #self.columnCombination.add(set())
        self.columnCombination1 = [] #список  (list)
        self.columnCombination1.append(set())

        
    
    def update(self, indexI, indexJ, word, row):
        #print('Dictionary delta-index word = ' + str(index - self.mapColNum.get(word, index)))
        self.row = row
        self.index = indexI
        #self.setsDict.add(word)
        #self.mapColNum.update([(word, index)])
        
        # попадаем в окно                  по умолчанию ставим indexI
        if(indexI - int(self.mapColNum.get(word, indexI)) < window ):
            # записываем слово в множество колонки
            self.setsDict.add(word)
            # обновляем/записываем номер строки в df для этого слова
            self.mapColNum.update([(word, indexI)])
            # записываем индекс слова для множества self.setsDict
            if(word not in self.mapIndexSets):
                self.mapIndexSets.update([(word, len(self.setsDict)-1)])
            #self.columnCombination[j].add(word)
        else:
            self.setsDict.remove(word) # выскочили из окна, удаляем элемент из множества
            del self.mapColNum[word]
            
        for j in range(len(row)): # самое главное здесь
            #print('row[' + str(j) + '] = ' + str(row[j]))
            #if(j != indexJ):
            if(j >= len(self.columnCombination)):
               self.columnCombination.append(set()) # добавляем нехватающий set
        
            self.columnCombination[j].add(row[j])
            
            
            
            
        # расширяем self.mapDictCorr, если требуется    
        if(len(self.setsDict) > len(self.mapDictCorr)):
            self.mapDictCorr.append(dict()) # добавляем недостающий dict
            #for m in range(len(row)):
                #self.mapDictCorr[self.mapIndexSets.get(word, "")].update({word:{row[m], 0}})

        for m in range(len(row)):
            #print(len(self.setsDict))
            #print(self.mapIndexSets.get(word, "")-1)
                        
#            print('\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
 #           print("{}{}".format("word   = ", word))
  #          print("{}{}{}{}".format('row[', str(m), '] = ', str(row[m])))
   #         #print("{}{}".format('self.mapIndexSets = ', self.mapIndexSets.keys()))
    #        print("{}{}".format('self.mapIndexSets2= ', self.mapIndexSets.items()))
     #       print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
      #      print("{}{}".format ('словарь1 = ', self.mapDictCorr[self.mapIndexSets.get(word, "")]))
        
            # ключ row[m], если ключем взять word, то все будет каждый цикл сбрасываться
            #    массив словарей |    значение - индекс  | число повторений по ключу
            num1  = self.mapDictCorr[self.mapIndexSets.get(word, "")].get(row[m], {})
            if(num1 != {}):
                num2 = num1.get(word, 0)
                num3 = int(num2) + 1 # увеличиваем число повторений
                #print("{}{}".format('число совпадений = ', num1))
                # массив словарей | значение - индекс  | обновление числа повторений
                self.mapDictCorr[self.mapIndexSets.get(word)].update({row[m]:{word:num3}})
                #print("{}{}".format ('словарь2 = ', self.mapDictCorr[self.mapIndexSets.get(word, "")]))  #.update([(row[m], num+1)])
            else:
                self.mapDictCorr[self.mapIndexSets.get(word)].update({row[m]:{word:1}})
                        
            
           

    def setPowerSets(self):
        self.powerSets = len(self.setsDict)
        
        
    
                
    def print1(self):
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print("{}{}".format ('список слов в колонке = ',self.setsDict))
        print("{}{}".format ('\nслово - индекс = ',self.mapIndexSets))
        #print("{}{}".format ('количество совпадений с другими колонками = ',self.mapDictCorr))
        for i in range(len(self.row)):
            print("{}{}{}{}".format ('\nself.mapDictCorr[col=',i,'] = ',self.mapDictCorr[i]))
        print('?????????????????????????????????????\n')    

        
        

Это просто тест.

In [34]:
lotsosets=[set() for i in range(5)]
lotsosets
#[set([]), set([]), set([]), set([]), set([])]
lotsosets[0].add('see me?')
lotsosets
#[set(['see me?']), set([]), set([]), set([]), set([])]
lotsosets[1].add('imma here too')
lotsosets
#[set(['see me?']), set(['imma here too']), set([]), set([]), set([])]

[{'see me?'}, {'imma here too'}, set(), set(), set()]

In [35]:
window = 20

# создаем и инициализируем myRow (экземпляры класса Column)
myRow = [None] * len(df.columns)   # Make a list of columns
for j in range(len(df.columns)):   # инициализация
    myRow[j] = Column(len(df.columns))

    
#df.iloc[0,1]
for i in range(len(df)):             # прошлись по каждой строке
    #df.iloc[[i]]
    #print("№" + str(i))
    for j in range(len(df.columns)): # прошлись по каждой колонке
        columnJ = myRow[j]    #  элемент       строка
        columnJ.update(i, j, df.iloc[i,j], df.iloc[i])#df.iloc[i,j] - элемент datafram'a
        columnJ.setPowerSets()
        myRow[j] = columnJ    # update row[j]
        #print ('Dictionary index = ' + str(columnJ.mapDictCol.get(df.iloc[i,j])))


# printing row[j]
for j in range(len(df.columns)):
    print('{}{}{}'.format('myRow[', j, ']')) # сами элемены множества (это колонка)
    print('powerSets = ' + str(myRow[j].powerSets))
    
    ##print(myRow[j].mapColNum.keys())
    #print(myRow[j].mapColNum.values()) # индексы, где эл-ты множества последний раз записаны в Sets
    ##print(myRow[j].mapColNum.items())
    
    myRow[j].print1()
    
     

myRow[0]
powerSets = 11
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
список слов в колонке = {130, 140, 111, 112, 113, 114, 151, 152, 121, 122, 123}

слово - индекс = {111: 0, 112: 1, 113: 2, 114: 3, 121: 4, 122: 5, 123: 6, 130: 7, 140: 8, 151: 9, 152: 10}

self.mapDictCorr[col=0] = {111: {111: 8}, 'Education, Level Unspecified': {111: 4}, 'Education, Niveau non spécifié': {111: 4}, 11110: {111: 1}, nan: {111: 1}, 'Education policy and administrative management': {111: 1}, 'Education sector policy, planning and programmes; aid to education ministries, administration and management systems; institution capacity building and advice; school management and governance; curriculum and materials development; unspecified education activities.': {111: 1}, 'Politique de l’éducation et gestion administrative': {111: 1}, 'Politique de l’éducation, planification et programmes\xa0; aide aux ministères de l’éducation, à l’administration et au développement de systèmes de gestion, renforcement des capacités 

In [37]:
#type(row[3].columnCombination) # это list
#row[2].columnCombination[6]    # row[2] - вторая колонка
                                # columnCombination[6] - 6-ой элемент множества в  row[2]
                                # взаимодейтвует c этими элементами из соседних колонок
        
#for k in range(len(df.columns)):
 #   print('\n!!!!!!!!!!!!!! колонка ' + str(k) + '\n')
  #  for n in range(len(myRow[k].columnCombination)):
   #     print('   сочетание колонки ' + str(k) + ' (-корреляция) с колонкой__ ' + str(n))
    #    print(len(myRow[k].columnCombination[n]))
                
#print(dir(row[k].columnCombination[n])) 
#type(row[k].columnCombination)

array12 = np.arange(100).reshape(10,10)

for m in range(10):
    for n in range(10):
        array12[m, n] = len(myRow[m].columnCombination[n])


print('\n')
print(array12)
print('\n\n')

array123 = np.arange(100).reshape(10,10)

for k in range(len(df.columns)):
    print('{}{}'.format('\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! колонка № ', k)) 
    for l in myRow[k].mapDictCorr[k].keys():
        l1 = 0
        for l2 in range(len(myRow)):
            if(l in myRow[l2].setsDict):
                l1 = l2
                #print ("{}{}{}{}".format('\n!!!!!!l1 = ', l1, ' !!!!!!!l = ', l))
        print ("{}{}{}{}".format('\nl (колонка № ', l1, ') = ', l))
        #print(myRow[k].mapDictCorr[k][l])
        for m in myRow[k].mapDictCorr[k][l].keys():
            print ("{}{}{}{}".format('m (колонка № ', k, ') = ', m))
            print ("{}{}".format('число   совпадений = ',myRow[k].mapDictCorr[k][l][m]))
            print ("{}{}".format('мощность множества = ',myRow[k].powerSets))
            array123[l1][k] = myRow[k].mapDictCorr[k][l][m]
    

print('\n')
print(array123)
print('\n\n')
#for i in range(len(df.columns)):
 #   print('{}{}'.format('\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! колонка № ', i)   ) 
    #for j in range(len(df.columns)):
        #print("{}{}{}{}".format ('\nself.mapDictCorr[col=',i,'] = ',myRow[i].mapDictCorr[j]))
        #print('?????????????????????????????????????\n')
        
        #print(myRow[i].mapDictCorr[j].items())






[[ 11  11  11  11  68 100 100  99 100  99]
 [ 11  11  11  11  68 100 100  99 100  99]
 [ 11  11  11  11  68 100 100  99 100  99]
 [ 11  11  11  11  68 100 100  99 100  99]
 [ 11  11  11  11  68 100 100  99 100  99]
 [ 11  11  11  11  68 100 100  99 100  99]
 [ 11  11  11  11  68 100 100  99 100  99]
 [ 11  11  11  11  68 100 100  99 100  99]
 [ 11  11  11  11  68 100 100  99 100  99]
 [ 11  11  11  11  68 100 100  99 100  99]]




!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! колонка № 0

l (колонка № 3) = 111
m (колонка № 0) = 111
число   совпадений = 8
мощность множества = 11

l (колонка № 1) = Education, Level Unspecified
m (колонка № 0) = 111
число   совпадений = 4
мощность множества = 11

l (колонка № 2) = Education, Niveau non spécifié
m (колонка № 0) = 111
число   совпадений = 4
мощность множества = 11

l (колонка № 4) = 11110
m (колонка № 0) = 111
число   совпадений = 1
мощность множества = 11

l (колонка № 0) = nan
m (колонка № 0) = 111
число   совпадений = 1
мощность множества = 11